In [6]:
import sys
!{sys.executable} -m pip install -r requirements.txt

You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.


In [7]:
from unet.model import *
from unet.data import *

Train Unet with EAT Data
----

### Train with data generator

In [8]:
data_gen_args = dict(rotation_range=0.2,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     shear_range=0.05,
                     zoom_range=0.05,
                     horizontal_flip=True,
                     fill_mode='nearest')
generator = trainGenerator(8,'../Training Data','AllImages','AllMasks',data_gen_args,save_to_dir = None, image_color_mode = 'rgb')
model = unet()
model_checkpoint = ModelCheckpoint('unet_EAT.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(generator,steps_per_epoch=90,epochs=300,callbacks=[model_checkpoint])

<ipython-input-8-175ece19ead6>:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator,steps_per_epoch=90,epochs=300,callbacks=[model_checkpoint])


Found 1567 images belonging to 1 classes.
Found 1567 images belonging to 1 classes.
Epoch 1/300
90/90 [==============================] - ETA: 0s - loss: 129.3681 - accuracy: 0.9617 - auc: 0.8278
Epoch 00001: loss improved from inf to 129.36810, saving model to unet_EAT.hdf5
90/90 [==============================] - 34s 355ms/step - loss: 129.3681 - accuracy: 0.9617 - auc: 0.8278
Epoch 2/300
90/90 [==============================] - ETA: 0s - loss: 0.1032 - accuracy: 0.9640 - auc: 0.9184
Epoch 00002: loss improved from 129.36810 to 0.10322, saving model to unet_EAT.hdf5
90/90 [==============================] - 32s 351ms/step - loss: 0.1032 - accuracy: 0.9640 - auc: 0.9184
Epoch 3/300
90/90 [==============================] - ETA: 0s - loss: 0.1179 - accuracy: 0.9614 - auc: 0.9109
Epoch 00003: loss did not improve from 0.10322
90/90 [==============================] - 28s 317ms/step - loss: 0.1179 - accuracy: 0.9614 - auc: 0.9109
Epoch 4/300
90/90 [==============================] - ETA: 0s -

KeyboardInterrupt: 

Test Model
----

In [14]:
def generateTestsFromPath(test_path, target_size = (256,256)):
    images = os.listdir(test_path)
    for i in images:
        img = io.imread(test_path +"/" +i, as_gray = False)
        img = img / 255
        img = trans.resize(img,(256,256))
        img = img[:,:,0:3]
        img = np.reshape(img,(1,)+img.shape)
        yield img

def saveResults(results,save_path):
    shape = results.shape
    for i in range(0,shape[0]):
        result = results[i,:,:,0] * ((2**8)-1)
        resultRGB = np.zeros((shape[1],shape[2],3))
        resultRGB[:,:,0] = result
        resultRGB[:,:,1] = result
        resultRGB[:,:,2] = result
        matplotlib.image.imsave(os.path.join(save_path,"%d_predict.png"%i), resultRGB.astype(np.uint8))

gen = generateTestsFromPath("../Test Data/AllImages")
model = unet()
model.load_weights("unet_EAT.hdf5")
results = model.predict_generator(gen,verbose=1)
saveResults(results, "../Test Data/AllMasks")

<ipython-input-14-cc6db0cafe29>:24: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  results = model.predict_generator(gen,verbose=1)


774/774 [==============================] - 36s 46ms/step


In [12]:
shape = results.shape
result = results[0,:,:,0] * ((2**8)-1)
resultRGB = np.zeros((shape[1],shape[2],3))
resultRGB[:,:,0] = result
resultRGB[:,:,1] = result
resultRGB[:,:,2] = result
resultRGB.shape

(256, 256, 3)

In [13]:
import matplotlib.pylab as plt
import skimage.io as io

model = unet()
model.load_weights("unet_EAT.hdf5")
test_path = "../Test Data/AllImages/15.png"

img = io.imread(test_path,as_gray = False)
img = img / 255
img = trans.resize(img,(256,256))
img = img[:,:,0:3]
img = np.reshape(img,(1,)+img.shape)
result = model.predict(img)

In [11]:
plt.imshow(result[0,:,:,0])
plt.show()

IndexError: too many indices for array: array is 2-dimensional, but 4 were indexed